# Clase 15: Ejemplo modelo estocástico de 2 etapas: el problema Dakota (en Julia)

 Prof. Tito Homem-de-Mello

In [ ]:
#import Pkg; Pkg.add("JuMP");
#import Pkg; Pkg.add("HiGHS"); 
#import Pkg; Pkg.add("Statistics");
#import Pkg; Pkg.add("Random");
#import Pkg; Pkg.add("Distributions");
#using Pkg; Pkg.add("DataFrames") 

In [ ]:
using JuMP
using HiGHS
using Statistics
using Random
using Distributions
using DataFrames

## Carguemos los datos

### Tabla con costos y precios

In [ ]:
insumos = DataFrame(MatPrima=["Madera","Acabado","Carpinteria"],
                       Costo=[2,4,5.2], Escritorios=[8,4,2], Mesas=[6,2,1.5], Sillas=[1,1.5,0.5])

precios = DataFrame(Escritorios=60, Mesas=40, Sillas=10);

In [ ]:
insumos

In [ ]:
precios

### Vector de escenarios posibles de demanda (el último valor corresponde al promedio de los anteriores)


In [ ]:
D = DataFrame(Escenario="Bajo", Escritorios=50, Mesas=20, Sillas=180)
push!(D, ["Medio"  150  105  220])
push!(D, ["Alto"   250  250  500])
push!(D, ["Promedio" 150 125 300])

println("Demanda:")
println(D)

## Inicialmente resolvemos el problema para 1 escenario (el escenario promedio)


In [ ]:
D_e = D[4, :Escritorios] #Escenario=="Promedio"
D_m = D[4, :Mesas] #Escenario=="Promedio"
D_s = D[4, :Sillas] #Escenario=="Promedio"

p_e = precios[1,:Escritorios]   
p_m = precios[1,:Mesas]  
p_s = precios[1,:Sillas] 

c_m = insumos[1, :Costo]  #MatPrima=="Madera"
c_a = insumos[2, :Costo]  #MatPrima=="Acabado"
c_c = insumos[3,:Costo]  #MatPrima=="Carpinteria"

ins_mad = insumos[1, [:Escritorios, :Mesas, :Sillas]]   #MatPrima=="Madera"
ins_aca = insumos[2, [:Escritorios, :Mesas, :Sillas]]   #MatPrima=="Acabado"
ins_car = insumos[3, [:Escritorios, :Mesas, :Sillas]];   #MatPrima=="Carpinteria"



In [ ]:
model_det = Model(HiGHS.Optimizer)

In [ ]:
@variable(model_det, x_m >= 0);
@variable(model_det, x_a >= 0);
@variable(model_det, x_c >= 0);
@variable(model_det, y_e >= 0);
@variable(model_det, y_m >= 0);
@variable(model_det, y_s >= 0);
@constraint(model_det, x_m >= ins_mad[1]*y_e + ins_mad[2]*y_m + ins_mad[3]*y_s)
@constraint(model_det, x_a >= ins_aca[1]*y_e + ins_aca[2]*y_m + ins_aca[3]*y_s)
@constraint(model_det, x_c >= ins_car[1]*y_e + ins_car[2]*y_m + ins_car[3]*y_s)
@constraint(model_det, y_e <= D_e) 
@constraint(model_det, y_m <= D_m) 
@constraint(model_det, y_s <= D_s) 
 
@objective(model_det, Max, p_e*y_e + p_m*y_m + p_s*y_s - c_m*x_m - c_a*x_a - c_c*x_c);
optimize!(model_det);

In [ ]:
etapa1_det = DataFrame(Tipo="Insumos",
                     Madera=value(x_m),
                     Acabado=value(x_a), 
                     Carpinteria=value(x_c))

etapa2_det = DataFrame(Tipo="Producción", 
                     Escritorios=value(y_e),
                     Mesas=value( y_m), 
                     Sillas=value(y_s))

println("\nInsumos solución determinista:\n")
println(etapa1_det)

println("\nProducción planificada:\n")
println(etapa2_det)

println("\n \n  Valor óptimo modelo determinista:= ", objective_value(model_det))

## Resolvamos ahora el caso *estocástico*:

In [ ]:
ns = 3;   #Nro. de  escenarios

In [ ]:
model_est = Model(HiGHS.Optimizer);

In [ ]:
@variable(model_est, x_m >= 0);
@variable(model_est, x_a >= 0);
@variable(model_est, x_c >= 0);
@variable(model_est, y_e[i=1:ns] >= 0);
@variable(model_est, y_m[i=1:ns] >= 0);
@variable(model_est, y_s[i=1:ns] >= 0);
@constraint(model_est, [i in 1:ns], x_m >= ins_mad[1]*y_e[i] + ins_mad[2]*y_m[i] + ins_mad[3]*y_s[i])
@constraint(model_est, [i in 1:ns], x_a >= ins_aca[1]*y_e[i] + ins_aca[2]*y_m[i] + ins_aca[3]*y_s[i])
@constraint(model_est, [i in 1:ns], x_c >= ins_car[1]*y_e[i] + ins_car[2]*y_m[i] + ins_car[3]*y_s[i])
@constraint(model_est, [i in 1:ns], y_e[i] <= D[i,2]) 
@constraint(model_est, [i in 1:ns], y_m[i] <= D[i,3]) 
@constraint(model_est, [i in 1:ns], y_s[i] <= D[i,4]) 
 
@objective(model_est, Max, (1/ns)*sum(p_e*y_e[i]+ p_m*y_m[i]+ p_s*y_s[i] for i in 1:ns)
                            - c_m*x_m - c_a*x_a - c_c*x_c);
optimize!(model_est);

### Veamos la solución optima para 1ra etapa y 2da etapa (la solución de 2da etapa es para cada escenario)

In [ ]:
scenarios = D[:, [:1]]

etapa1_est = DataFrame(Tipo="Insumos",
                     Madera=value(x_m),
                     Acabado=value(x_a), 
                     Carpinteria=value(x_c))

ye = JuMP.value.(y_e[:])      
ym = JuMP.value.(y_m[:])  
ys = JuMP.value.(y_s[:])  

etapa2_est  = DataFrame(Escenario=scenarios[1,1],
                       Escritorios=ye[1],
                       Mesas=ym[1], 
                       Sillas=ys[1])
for i in 2:ns
    push!(etapa2_est, [scenarios[i,1]  ye[i]  ym[i] ys[i]])
end

println("\nInsumos solución estocástica:\n")
println(etapa1_est)

println("\nProducción planificada:\n")
println(etapa2_est)


println("\n \n Utilidad esperada de la solución estocástica: = ",objective_value(model_est))

### Si queremos ver el valor de la función objetivo para cada etapa, podemos hacer el cálculo manual:

In [ ]:
s1 = - (collect(Float64, etapa1_est[1,2:4])' * insumos[:, :Costo]);

s2 =( (collect(Float64, etapa2_est[1,2:4]))' * collect(Float64, precios[:1, :]) +
     (collect(Float64, etapa2_est[2,2:4]))' * collect(Float64, precios[:1, :]) +
     (collect(Float64, etapa2_est[3,2:4]))' * collect(Float64, precios[:1, :]))/3
obj_est = s1+s2


println("Utilidad esperada de la solución estocástica: ", s1," + ",s2," = " ,obj_est)

## ¿Cómo se compara la solución estocástica con la determinista?

### Recordemos la solución determinista:

In [ ]:
println("\nInsumos solución determinista::\n")
println(etapa1_det)

println("\nProducción planificada:\n")
println(etapa2_det)

### Qué se observa?

### - La solución determinista usa bastante más insumos

### - Pero la producción en esa solución es *infactible* para los escenarios de baja y media demanda!

### Sin embargo, podemos comparar la solución de 1ra etapa de los modelos determinista y estocástico.

### ¿Cómo?

### Hay que evaluar la función objetivo $𝑐^𝑇 𝑥 + 𝐸[𝑄(𝑥, D)]$  para $x^{DET}=$sol. determinista y $x^{EST}=$sol. estocástica.

### Una manera de hacerlo es resolver otra vez el problema estocástico, pero agregando la restricción $x==x^{DET}$

In [ ]:
xdet = collect(Float64, etapa1_det[1,2:4]);
xm_det = xdet[1];
xa_det = xdet[2];
xc_det = xdet[3];

In [ ]:
@constraint(model_est, x_m==xm_det) 
@constraint(model_est, x_a==xa_det) 
@constraint(model_est, x_c==xc_det) 
optimize!(model_est);

### Veamos cuál fue la producción *real* correspondiente a la solución de 1ra etapa determinista.

In [ ]:
scenarios = D[:, [:1]]


ye = JuMP.value.(y_e[:])      
ym = JuMP.value.(y_m[:])  
ys = JuMP.value.(y_s[:])  

etapa2_det_real  = DataFrame(Escenario=scenarios[1,1],
                       Escritorios=ye[1],
                       Mesas=ym[1], 
                       Sillas=ys[1])
for i in 2:ns
    push!(etapa2_det_real , [scenarios[i,1]  ye[i]  ym[i] ys[i]])
end

println("\nInsumos solución determinista:\n")
println(etapa1_det)

println("\nProducción REAL:\n")
println(etapa2_det_real )





### La producción de silla es fraccionaria! ¿Hace sentido?

### ¿Cómo evitar eso?

### Hay que agregar restriccciones de *integralidad* a las variables de 2da etapa:

In [ ]:
model_int = Model(HiGHS.Optimizer);

In [ ]:
@variable(model_int, x_m >= 0);
@variable(model_int, x_a >= 0);
@variable(model_int, x_c >= 0);
@variable(model_int, y_e[i=1:ns] >= 0, integer=true);
@variable(model_int, y_m[i=1:ns] >= 0, integer=true);
@variable(model_int, y_s[i=1:ns] >= 0, integer=true);
@constraint(model_int, [i in 1:ns], x_m >= ins_mad[1]*y_e[i] + ins_mad[2]*y_m[i] + ins_mad[3]*y_s[i])
@constraint(model_int, [i in 1:ns], x_a >= ins_aca[1]*y_e[i] + ins_aca[2]*y_m[i] + ins_aca[3]*y_s[i])
@constraint(model_int, [i in 1:ns], x_c >= ins_car[1]*y_e[i] + ins_car[2]*y_m[i] + ins_car[3]*y_s[i])
@constraint(model_int, [i in 1:ns], y_e[i] <= D[i,2]) 
@constraint(model_int, [i in 1:ns], y_m[i] <= D[i,3]) 
@constraint(model_int, [i in 1:ns], y_s[i] <= D[i,4]) 
@constraint(model_int, x_m==xm_det) 
@constraint(model_int, x_a==xa_det) 
@constraint(model_int, x_c==xc_det) 
@objective(model_int, Max, (1/ns)*sum(p_e*y_e[i]+ p_m*y_m[i]+ p_s*y_s[i] for i in 1:ns)
                            - c_m*x_m - c_a*x_a - c_c*x_c);
optimize!(model_int);

### Resultados para el modelo determinista con variables enteras:

In [ ]:
scenarios = D[:, [:1]]


ye = JuMP.value.(y_e[:])      
ym = JuMP.value.(y_m[:])  
ys = JuMP.value.(y_s[:])  

etapa2_det_int  = DataFrame(Escenario=scenarios[1,1],
                       Escritorios=ye[1],
                       Mesas=ym[1], 
                       Sillas=ys[1])
for i in 2:ns
    push!(etapa2_det_int , [scenarios[i,1]  ye[i]  ym[i] ys[i]])
end

println("\nInsumos solución determinista:\n")
println(etapa1_det)

println("\nProducción REAL:\n")
println(etapa2_det_int )

println("\n \nUtilidad esperada de la solución determinista: = ",objective_value(model_int))


### Vemos que la solución estocástica es bastante superior!

### La diferencia se llama *Valor de la Solución Estocástica* (VSE).

### Podemos medir el VSE en porcentaje:

In [ ]:
VSE  = (objective_value(model_est)-objective_value(model_int))/abs(objective_value(model_int))*100
println("VSE = ",VSE)

### Podemos repetir el procedimiento de evaluación de soluciones para cualquier solución fija de 1ra etapa.

# Resolviendo el problema Dakota con muestreo

## Asumamos que la demanda es Normal con los mismos promedios de antes, y desviación estándar = 15

## Utilizaremos $N=30$ muestras de la demanda:

In [ ]:
ns = 30

### Cálculo de la cota inferior

In [ ]:
M = 20
Distr_e = Normal(150,15)
Distr_m = Normal(125,15)
Distr_s = Normal(300,15)

v = zeros(M+1)
for k in 1:M+1
    D = DataFrame(Escenario=1:ns,
                  Escritorios=round.(rand(Distr_e, ns)), #Notar la función "round" - queremos valores enteros
                  Mesas=round.(rand(Distr_m, ns)),
                  Sillas=round.(rand(Distr_s, ns)))
                  
    model_int = Model(HiGHS.Optimizer);
    @variable(model_int, x_m >= 0);
    @variable(model_int, x_a >= 0);
    @variable(model_int, x_c >= 0);
    @variable(model_int, y_e[i=1:ns] >= 0, integer=true);
    @variable(model_int, y_m[i=1:ns] >= 0, integer=true);
    @variable(model_int, y_s[i=1:ns] >= 0, integer=true);
    @constraint(model_int, [i in 1:ns], x_m >= ins_mad[1]*y_e[i] + ins_mad[2]*y_m[i] + ins_mad[3]*y_s[i])
    @constraint(model_int, [i in 1:ns], x_a >= ins_aca[1]*y_e[i] + ins_aca[2]*y_m[i] + ins_aca[3]*y_s[i])
    @constraint(model_int, [i in 1:ns], x_c >= ins_car[1]*y_e[i] + ins_car[2]*y_m[i] + ins_car[3]*y_s[i])
    @constraint(model_int, [i in 1:ns], y_e[i] <= D[i,2]) 
    @constraint(model_int, [i in 1:ns], y_m[i] <= D[i,3]) 
    @constraint(model_int, [i in 1:ns], y_s[i] <= D[i,4]) 
    @objective(model_int, Max, (1/ns)*sum(p_e*y_e[i]+ p_m*y_m[i]+ p_s*y_s[i] for i in 1:ns)
                                - c_m*x_m - c_a*x_a - c_c*x_c);
    optimize!(model_int);   
    
    v[k] = objective_value(model_int);
end


In [ ]:
v

### Cálculo de la cota superior; tomamos la solución del último problema ($M+1$)

In [ ]:
etapa1_est = DataFrame(Tipo="Insumos",
                     Madera=value(x_m),
                     Acabado=value(x_a), 
                     Carpinteria=value(x_c))
                     
xfix = collect(Float64, etapa1_est[1,2:4]);
xm_fix = xfix[1];
xa_fix = xfix[2];
xc_fix = xfix[3];

D = DataFrame(Escenario=1:ns,
              Escritorios=round.(rand(Distr_e, ns)), #Notar la función "round" - queremos valores enteros
              Mesas=round.(rand(Distr_m, ns)),
              Sillas=round.(rand(Distr_s, ns)))

model_int = Model(HiGHS.Optimizer);
@variable(model_int, x_m >= 0);
@variable(model_int, x_a >= 0);
@variable(model_int, x_c >= 0);
@variable(model_int, y_e[i=1:ns] >= 0, integer=true);
@variable(model_int, y_m[i=1:ns] >= 0, integer=true);
@variable(model_int, y_s[i=1:ns] >= 0, integer=true);
@constraint(model_int, [i in 1:ns], x_m >= ins_mad[1]*y_e[i] + ins_mad[2]*y_m[i] + ins_mad[3]*y_s[i])
@constraint(model_int, [i in 1:ns], x_a >= ins_aca[1]*y_e[i] + ins_aca[2]*y_m[i] + ins_aca[3]*y_s[i])
@constraint(model_int, [i in 1:ns], x_c >= ins_car[1]*y_e[i] + ins_car[2]*y_m[i] + ins_car[3]*y_s[i])
@constraint(model_int, [i in 1:ns], y_e[i] <= D[i,2]) 
@constraint(model_int, [i in 1:ns], y_m[i] <= D[i,3]) 
@constraint(model_int, [i in 1:ns], y_s[i] <= D[i,4]) 
@constraint(model_int, x_m==xm_fix) 
@constraint(model_int, x_a==xa_fix) 
@constraint(model_int, x_c==xc_fix) 
@objective(model_int, Max, (1/ns)*sum(p_e*y_e[i]+ p_m*y_m[i]+ p_s*y_s[i] for i in 1:ns)
                            - c_m*x_m - c_a*x_a - c_c*x_c);
optimize!(model_int);



In [ ]:
ye = JuMP.value.(y_e[:])      
ym = JuMP.value.(y_m[:])  
ys = JuMP.value.(y_s[:])  

s1 = - (xfix' * insumos[:, :Costo]);
p = collect(Float64, precios[:1, :])

val2 = [ye[1]  ym[1] ys[1]] * p;
for i in 2:ns
     u =  ( [ye[i]  ym[i] ys[i]] * p);
     val2 = [val2 u];
end




In [ ]:
val2

In [ ]:
println("Solución evaluada:\n \n")
etapa1_est = DataFrame(Madera=xm_fix,
                       Acabado=xa_fix, 
                       Carpinteria=xc_fix)

println(etapa1_est)

In [ ]:
println("\n Limite inferior = ", mean(v[2:M+1])-1.96*std(v[2:M+1])/sqrt(M),"\n")
println("\n Limite superior = ",s1+mean(val2)+1.96*std(val2)/sqrt(ns),"\n")